In [1]:
"""
Train a classifier on top of a language model trained with `pretrain_lm.py`.
Optionally fine-tune LM before.
"""
import numpy as np
import pickle

import torch
import torch.nn.functional as F
from fastai.text import TextLMDataBunch, TextClasDataBunch, language_model_learner, text_classifier_learner, LanguageLearner
from fastai import fit_one_cycle
from fastai_contrib.utils import PAD, UNK, read_clas_data, PAD_TOKEN_ID, DATASETS, TRN, VAL, TST, ensure_paths_exists, get_sentencepiece
from fastai.text.transform import Vocab
from fastai.metrics import accuracy, accuracy_thresh, fbeta

import fire
from collections import Counter
from pathlib import Path

import pandas as pd
import csv
from functools import partial
from sklearn.metrics import f1_score, precision_score, recall_score
import mlflow
import os

In [2]:
%load_ext autoreload
%autoreload 2

# NTCIR-13 MedWeb
http://research.nii.ac.jp/ntcir/permission/ntcir-13/perm-ja-MedWeb.html

## copy data

In [3]:
!ls /home/ubuntu/dev/download/data/MedWeb_TestCollection/

NTCIR-13_MedWeb_en_test.xlsx	  NTCIR-13_MedWeb_ja_training.xlsx  readme.txt
NTCIR-13_MedWeb_en_training.xlsx  NTCIR-13_MedWeb_zh_test.xlsx
NTCIR-13_MedWeb_ja_test.xlsx	  NTCIR-13_MedWeb_zh_training.xlsx


In [4]:
!cat /home/ubuntu/dev/download/data/MedWeb_TestCollection/readme.txt

NTCIR-13 MedWeb task: Test set

Training data
-MedWeb Japanese subtask train data: NTCIR-13_MedWeb_ja_training.xlsx
-MedWeb English subtask train data: NTCIR-13_MedWeb_en_training.xlsx
-MedWeb Chinese subtask train data: NTCIR-13_MedWeb_zh_training.xlsx

Test data
-MedWeb Japanese subtask test data: NTCIR-13_MedWeb_ja_test.xlsx
-MedWeb English subtask test data: NTCIR-13_MedWeb_en_test.xlsx
-MedWeb Chinese subtask test data: NTCIR-13_MedWeb_zh_test.xlsx


Columns in each sheet are as follows. 
ID: Pseudo tweet ID that corresponds to the corpora of other language (e.g., the tweet of ``1en'' corresponds to the tweets of ``1ja'' and ``1zh'') 
Tweet: Pseudo tweet message 
Influenza: P (Positive) / N (Negative)
Diarrhea: P (Positive) / N (Negative)
Hayfever: P (Positive) / N (Negative)
Cough: P (Positive) / N (Negative)
Headache: P (Positive) / N (Negative)
Fever: P (Positive) / N (Negative)
Runnynose: P (Positive) / N (Negative)
Cold: P (Positive) / N (Negative)

For details, please refer 

In [5]:
!mkdir -p data/MedWeb

In [6]:
!cp /home/ubuntu/dev/download/data/MedWeb_TestCollection/*ja* data/MedWeb/

In [7]:
!ls data/MedWeb/

lbl_names.pkl  NTCIR-13_MedWeb_ja_test.xlsx	 tmp	    valid_ch.txt
models	       NTCIR-13_MedWeb_ja_training.xlsx  train.csv  valid.csv


## read data

### train

In [8]:
trn = pd.read_excel('data/MedWeb/NTCIR-13_MedWeb_ja_training.xlsx',sheet_name='ja_train')

In [9]:
trn.head(100)

,ID,Tweet,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold
0,1ja,風邪を引くと全身がだるくなる。,n,n,n,n,n,n,n,p
1,2ja,花粉症の症状が出てきたのは久し振りだ。,n,n,p,n,n,n,p,n
2,3ja,花粉症のせいでずっと微熱でぼーっとしてる。眠い。,n,n,p,n,n,p,p,n
3,4ja,薬飲んだけど鼻水おさまる気配なし,n,n,n,n,n,n,p,n
4,5ja,ネパールに旅行に行った際に下痢になって大変だったよ。,n,n,n,n,n,n,n,n
5,6ja,咳くらいで休むのはゆとりだけだろ。どんなときでも仕事にでるのは大事だ。,n,n,n,p,n,n,n,n
6,7ja,鼻づまりで今日は休むわー,n,n,n,n,n,n,p,n
7,8ja,まさか花粉症になるとは。,n,n,p,n,n,n,p,n
8,9ja,熱は出てるけどお腹に来る風邪じゃなさそう。,n,n,n,n,n,p,n,p
9,10ja,痰に血が混じってきもい,n,n,n,p,n,n,n,n


In [10]:
# name of labels
lbl_names = trn.columns[2:]
lbl_names

Index(['Influenza', 'Diarrhea', 'Hayfever', 'Cough', 'Headache', 'Fever',
       'Runnynose', 'Cold'],
      dtype='object')

In [11]:
# save
pickle.dump(lbl_names, open('data/MedWeb/lbl_names.pkl','wb'))

3列目以降がpの列名を列挙して、ラベルとする

trn_lbl = ['_'.join(row) for row in ((trn.iloc[:,2:] == 'p')*1).values.astype(str)]

trn_lbl = [','.join(row) for row in trn.columns[2:] + '_' +trn.iloc[:,2:]]

for i, row in trn.iterrows():
    print(','.join(lbl_names[row[2:]=='p']))
    if i > 5:
        break

trn_lbl = []
for i, row in trn.iterrows():
    trn_lbl.append(lbl_idx[(row[2:]=='p').values])

In [12]:
lbl_idx = np.asarray([i for i in range(len(lbl_names))])

In [13]:
lbl_idx

array([0, 1, 2, 3, 4, 5, 6, 7])

In [14]:
trn_lbl = []
for i, row in trn.iterrows():
    trn_lbl.append(lbl_names[(row[2:]=='p')].tolist())

In [15]:
trn_lbl[:5]

[['Cold'],
 ['Hayfever', 'Runnynose'],
 ['Hayfever', 'Fever', 'Runnynose'],
 ['Runnynose'],
 []]

In [16]:
# put data into df and save
trn_df = pd.DataFrame({'text':trn['Tweet'].values, 'labels':trn_lbl},columns=['labels', 'text'])
trn_df.to_csv('data/MedWeb/train.csv', header=False, index=False)


In [17]:
trn_df

,labels,text
0,[Cold],風邪を引くと全身がだるくなる。
1,"[Hayfever, Runnynose]",花粉症の症状が出てきたのは久し振りだ。
2,"[Hayfever, Fever, Runnynose]",花粉症のせいでずっと微熱でぼーっとしてる。眠い。
3,[Runnynose],薬飲んだけど鼻水おさまる気配なし
4,[],ネパールに旅行に行った際に下痢になって大変だったよ。
5,[Cough],咳くらいで休むのはゆとりだけだろ。どんなときでも仕事にでるのは大事だ。
6,[Runnynose],鼻づまりで今日は休むわー
7,"[Hayfever, Runnynose]",まさか花粉症になるとは。
8,"[Fever, Cold]",熱は出てるけどお腹に来る風邪じゃなさそう。
9,[Cough],痰に血が混じってきもい


### test

In [18]:
tst = pd.read_excel('data/MedWeb/NTCIR-13_MedWeb_ja_test.xlsx',sheet_name='ja_test')

In [19]:
tst.head(100)

,ID,Tweet,Influenza,Diarrhea,Hayfever,Cough,Headache,Fever,Runnynose,Cold
0,1921ja,旅行に行ったら、土産にインフルもらってきた。,p,n,n,n,n,p,n,n
1,1922ja,きつい上司、頭痛の種,n,n,n,n,n,n,n,n
2,1923ja,もう誰か翻訳してくれないときつい、宇宙の言葉ですかってくらい通じなくて頭痛してきた。,n,n,n,n,n,n,n,n
3,1924ja,インフル感染の危機。,p,n,n,n,n,p,n,n
4,1925ja,鼻づまりがひどいからスピーチは無理だ,n,n,n,n,n,n,p,n
5,1926ja,頭痛がしているので帰ることにする。,n,n,n,n,p,n,n,n
6,1927ja,インフルエンザになって部活のみんなから爆笑されたよ,p,n,n,n,n,p,n,n
7,1928ja,下痢って英語でなんていうんだろう。,n,n,n,n,n,n,n,n
8,1929ja,スペイン風邪、香港風邪っていうけど、日本風邪ってあるの？,n,n,n,n,n,n,n,n
9,1930ja,犬って鼻づまりとかするのかな？,n,n,n,n,n,n,n,n


In [20]:
tst_lbl = []
for i, row in tst.iterrows():
    tst_lbl.append(lbl_names[(row[2:]=='p')].tolist())

In [21]:
tst_lbl[:5]

[['Influenza', 'Fever'], [], [], ['Influenza', 'Fever'], ['Runnynose']]

In [22]:
# name of labels
lbl_names = tst.columns[2:]
lbl_names

Index(['Influenza', 'Diarrhea', 'Hayfever', 'Cough', 'Headache', 'Fever',
       'Runnynose', 'Cold'],
      dtype='object')

In [23]:
# put data into df and save
tst_df = pd.DataFrame({'text':tst['Tweet'].values, 'labels':tst_lbl},columns=['labels', 'text'])
# change name from test to valid (fastai lm expects valid)
tst_df.to_csv('data/MedWeb/valid.csv', header=False, index=False)


In [24]:
tst_df.head()

,labels,text
0,"[Influenza, Fever]",旅行に行ったら、土産にインフルもらってきた。
1,[],きつい上司、頭痛の種
2,[],もう誰か翻訳してくれないときつい、宇宙の言葉ですかってくらい通じなくて頭痛してきた。
3,"[Influenza, Fever]",インフル感染の危機。
4,[Runnynose],鼻づまりがひどいからスピーチは無理だ


## prepare data for lm fine tuning and classification
basically copying from train_cls.py

In [25]:
data_dir='data'
lang='ja' 
cuda_id=0 
pretrain_name='wt-100' 
model_dir='data/wiki/ja-100/models'
max_vocab=8000
name='MedWeb-clas'
dataset='MedWeb' 
frac_ds=1.0
spm_dir = 'data/wiki/ja/'

In [26]:
data_dir = Path(data_dir)
assert data_dir.name == 'data',\
    f'Error: Name of data directory should be data, not {data_dir.name}.'
dataset_dir = data_dir / dataset
model_dir = Path(model_dir)


In [27]:
if not torch.cuda.is_available():
    print('CUDA not available. Setting device=-1.')
    cuda_id = -1
torch.cuda.set_device(cuda_id)

print(f'Dataset: {dataset}. Language: {lang}.')

Dataset: MedWeb. Language: ja.


In [28]:
# here we're just loading the trained spm model
sp = get_sentencepiece(spm_dir, None, 'wt-all', vocab_size=max_vocab)

In [29]:
# load train, valid in df
train_df = pd.read_csv(dataset_dir/'train.csv',header=None)
valid_df = pd.read_csv(dataset_dir/'valid.csv',header=None)

In [30]:
train_df.head()

,0,1
0,['Cold'],風邪を引くと全身がだるくなる。
1,"['Hayfever', 'Runnynose']",花粉症の症状が出てきたのは久し振りだ。
2,"['Hayfever', 'Fever', 'Runnynose']",花粉症のせいでずっと微熱でぼーっとしてる。眠い。
3,['Runnynose'],薬飲んだけど鼻水おさまる気配なし
4,[],ネパールに旅行に行った際に下痢になって大変だったよ。


In [31]:
def strip_brackets(s):
    return s.strip('[]').replace("'","").replace(" ","")

In [32]:
train_df[0] = train_df[0].apply(strip_brackets)

In [33]:
valid_df[0] = valid_df[0].apply(strip_brackets)

In [34]:
train_df.head()

,0,1
0,Cold,風邪を引くと全身がだるくなる。
1,"Hayfever,Runnynose",花粉症の症状が出てきたのは久し振りだ。
2,"Hayfever,Fever,Runnynose",花粉症のせいでずっと微熱でぼーっとしてる。眠い。
3,Runnynose,薬飲んだけど鼻水おさまる気配なし
4,,ネパールに旅行に行った際に下痢になって大変だったよ。


### lm data

In [35]:
# set label_delim for multilabel (MultiCategory) data
data_lm = TextLMDataBunch.from_df(path=dataset_dir,train_df=train_df,valid_df=valid_df,label_delim=',',**sp)

### classification data

In [36]:
# set label_delim for multilabel (MultiCategory) data
data_clas = TextClasDataBunch.from_df(path=dataset_dir,train_df=train_df,valid_df=valid_df,label_delim=',',**sp)

In [37]:
data_clas.valid_ds.y.c2i

{'Cold': 0,
 'Cough': 1,
 'Diarrhea': 2,
 'Fever': 3,
 'Hayfever': 4,
 'Headache': 5,
 'Influenza': 6,
 'Runnynose': 7}

# LM fine tuning & Classification

In [38]:
def set_deterministic(deterministic=True):
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(0)
        np.random.seed(0)

In [39]:
def lm_fine_tuning( data_lm, lm_lr, bptt, emb_sz, nh, nl, qrnn,
                   pad_token,
                   pretrained_fnames, 
                   model_dir,
                   lm_enc_finetuned,
                   lm_drop_mult,
                   use_pretrained_lm,
                   fine_tune_lm,
                   deterministic,
                   lm_epochs=10,
                   lm_lr_discount=10**2
                  ):
    
    
    if not use_pretrained_lm:
        pretrained_fnames = None
    
    learn = language_model_learner(
        data_lm, bptt=bptt, emb_sz=emb_sz, nh=nh, nl=nl, qrnn=qrnn,
        pad_token=pad_token,
        pretrained_fnames=pretrained_fnames, 
        path=model_dir.parent, model_dir=model_dir.name,
        drop_mult=lm_drop_mult
    )
    
    set_deterministic(deterministic)
    
    if fine_tune_lm:
        print('Fine-tuning the language model...')
        learn.unfreeze()
        learn.fit(lm_epochs, slice(lm_lr/lm_lr_discount, lm_lr))
    #     learn.fit(10, slice(1e-3, 1e-1))
    else:
        print('Skipping fine tuning')

    # save fine tuned lm
    print(f"Saving models at {learn.path / learn.model_dir}")
    learn.save_encoder(lm_enc_finetuned)
    
    return learn

In [47]:
def classify_multilabel(data_clas, clas_lr, bptt, 
                        pad_token,
                        model_dir,
                        qrnn, emb_sz, nh, nl,
                        clas_drop_mult,
                        fine_tune_lm, use_lm,
                        cls_weights,
                        deterministic,
                        clas_lr_discount=2.6**4,
                        lin_ftrs=[50]
                       ):
    
    
    # change metric from accuracy to accuarcy_thresh and f1
    accuracy_multi =  partial(accuracy_thresh,thresh=0.5,sigmoid=True)
    f1 = partial(fbeta,thresh=0.5,beta=1,sigmoid=True) # corresponds to f1 score in sklearn with 'samples' option?

    print("Starting classifier training")
    learn = text_classifier_learner(data_clas, bptt=bptt, pad_token=pad_token,
                                  path=model_dir.parent, model_dir=model_dir.name,
                                  qrnn=qrnn, emb_sz=emb_sz, nh=nh, nl=nl,drop_mult=clas_drop_mult,
                                    lin_ftrs=lin_ftrs
                                    )
#     learn.model.reset()
    
    if use_lm:
        print('Loading language model')
        lm_enc = lm_enc_finetuned # if fine_tune_lm else lm_name
        learn.load_encoder(lm_enc)
    else:
        print('Training from scratch without language model')

    # change metric
    learn.metrics = [accuracy_multi,f1]

    # CRITICAL STEP
    #- need to adjust pos_weight of loss function to get the model working
    if cls_weights is not None:
        pos_weight = torch.cuda.FloatTensor(cls_weights[data_clas.train_ds.y.classes].values)
        bce_logits_weighted = partial(F.binary_cross_entropy_with_logits,  pos_weight=pos_weight)
        learn.loss_func = bce_logits_weighted

    # train
    set_deterministic(deterministic)
    
    learn.freeze_to(-1)
    learn.fit_one_cycle(1, clas_lr, moms=(0.8, 0.7), wd=1e-7)

    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(clas_lr / clas_lr_discount, clas_lr), moms=(0.8, 0.7), wd=1e-7)

    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(clas_lr / clas_lr_discount, clas_lr), moms=(0.8, 0.7), wd=1e-7)

    learn.unfreeze()
    learn.fit_one_cycle(2, slice(clas_lr /clas_lr_discount, clas_lr), moms=(0.8, 0.7), wd=1e-7)
    
    # results
    results={}
    results['accuracy_multi'] = learn.validate()[1]
    results['F1'] = learn.validate()[2]
    print(results)
    
    # save classifier
    print(f"Saving models at {learn.path / learn.model_dir}")
    learn.save(f'{model_name}_{name}')
    
    # prediction on validation data
    preds=learn.get_preds()

    # prediction
    p=preds[0]
    p = p.tolist()
    p = np.asarray(p)

    # binarize
    y_pred = (p>=0.5)*1.0


    # target
    t = preds[1].tolist()
    y_true = np.asarray(t)
    
    return y_true, y_pred

In [48]:
def evaluation(y_true,y_pred):
    ## metrics
    f1_micro = f1_score(y_true,y_pred,average='micro')
    f1_macro = f1_score(y_true,y_pred,average='macro')

    print('F1 score')
    print('micro: {}'.format(f1_micro))
    print('macro: {}'.format(f1_macro))

    precision_micro = precision_score(y_true,y_pred,average='micro')
    precision_macro = precision_score(y_true,y_pred,average='macro')

    print('\nPrecision score')
    print('micro: {}'.format(precision_micro))
    print('macro: {}'.format(precision_macro))

    recall_micro = recall_score(y_true,y_pred,average='micro')
    recall_macro = recall_score(y_true,y_pred,average='macro')

    print('\nRecall score')
    print('micro: {}'.format(recall_micro))
    print('macro: {}'.format(recall_macro))
    
    results = dict(
        f1_micro=f1_micro,
        f1_macro=f1_macro,
        precision_micro=precision_micro,
        precision_macro=precision_macro,
        recall_micro=recall_micro,
        recall_macro=recall_macro
    )
    
    return results

In [49]:
# Create Params dictionary
class Params(object):
    def __init__(self, qrnn,bs,bptt,emb_sz, nh, nl,fine_tune_lm,lm_lr,lm_drop_mult,
                 use_pretrained_lm,clas_lr,clas_drop_mult,use_lm,cls_weights,frac_ds,deterministic,lm_epochs,lm_lr_discount,clas_lr_discount,
                lin_ftrs):
        self.qrnn = qrnn
        self.fine_tune_lm = fine_tune_lm
        self.bs = bs
        self.bptt = bptt
        self.emb_sz = emb_sz
        self.nh = nh
        self.nl = nl
        self.lm_lr = lm_lr
        self.lm_drop_mult = lm_drop_mult
        self.use_pretrained_lm = use_pretrained_lm
        self.clas_lr = clas_lr
        self.clas_drop_mult = clas_drop_mult
        self.use_lm = use_lm
        self.cls_weights = cls_weights.values.tolist()
        self.frac_ds = frac_ds
        self.deterministic = deterministic 
        self.lm_epochs = lm_epochs 
        self.lm_lr_discount = lm_lr_discount     
        self.clas_lr_discount = clas_lr_discount 
        self.lin_ftrs = lin_ftrs                         

###  Parameters

In [61]:
# common to lm fine tuning and classification
qrnn=True
bs=20 
bptt=70
pad_token=PAD_TOKEN_ID

if qrnn:
    emb_sz, nh, nl = 400, 1550, 3
else:
    emb_sz, nh, nl = 400, 1150, 3
    
# lm fine tuning
fine_tune_lm=True 
lm_lr=5e-3
lm_lr_discount=10**2
lm_drop_mult=0.5
use_pretrained_lm=True
lm_epochs=20

# classification
clas_lr=2e-2
clas_lr_discount=2.6**4
clas_drop_mult=0.1
use_lm=True
lin_ftrs=[50*2]

# reduction of training data
frac_ds = 1.0

# reduce fluctuation between runs
deterministic = True

In [62]:
# class weights for classification loss function
# cls_weights = trn.shape[0]/(trn.iloc[:,2:]=='p').sum()/5
cls_weights = pd.Series(np.ones(8,)*1.0,index=trn.columns[2:])
cls_weights

Influenza    1.0
Diarrhea     1.0
Hayfever     1.0
Cough        1.0
Headache     1.0
Fever        1.0
Runnynose    1.0
Cold         1.0
dtype: float64

In [63]:
mlflow.set_experiment('20181213_MedWeb_vocab8k')

In [64]:
with mlflow.start_run():
    set_deterministic(deterministic)
    
    ## Log our parameters into mlflow
    args = Params(qrnn,bs,bptt,emb_sz, nh, nl,fine_tune_lm,lm_lr,lm_drop_mult,
                 use_pretrained_lm,clas_lr,clas_drop_mult,use_lm,cls_weights,frac_ds,deterministic,lm_epochs,lm_lr_discount,clas_lr_discount,
                 lin_ftrs)
    for key, value in vars(args).items():
        mlflow.log_param(key, value)
    
    ## create data set
    data_lm = TextLMDataBunch.from_df(path=dataset_dir,train_df=train_df.sample(frac=frac_ds,random_state=42),
                                      valid_df=valid_df,label_delim=',',**sp)
    data_clas = TextClasDataBunch.from_df(path=dataset_dir,train_df=train_df.sample(frac=frac_ds,random_state=42),
                                          valid_df=valid_df,label_delim=',',**sp)
    
    ## preprocess
    if qrnn:
        print('Using QRNNs...')
    model_name = 'qrnn' if qrnn else 'lstm'
    lm_name = f'{model_name}_{pretrain_name}'
    pretrained_fnames = (lm_name, f'itos_{pretrain_name}')

    ensure_paths_exists(data_dir,
                        dataset_dir,
                        model_dir,
                        model_dir/f"{pretrained_fnames[0]}.pth",
                        model_dir/f"{pretrained_fnames[1]}.pkl")
    lm_enc_finetuned  = f"{lm_name}_{dataset}_enc"

    ## fine tune lm
    p = None
    if use_lm:
        lm=lm_fine_tuning( data_lm, lm_lr, bptt, emb_sz, nh, nl, qrnn,
                          pad_token,
                          pretrained_fnames, 
                          model_dir,
                          lm_enc_finetuned,
                          lm_drop_mult,
                          use_pretrained_lm,
                          fine_tune_lm,
                          deterministic,
                          lm_epochs,
                          lm_lr_discount
                         )
        # check lm quality by predicting words after some text
        os.makedirs('tmp',exist_ok=True)
        p = lm.predict('料金が高い',n_words=200,no_unk=False)
        print(p)
        with open('tmp/lm_predicted.txt','w') as f:
            f.write(p)


    ## create classifier
    set_deterministic(deterministic)
    y_true, y_pred = classify_multilabel(data_clas, clas_lr, bptt, 
                                         pad_token,
                                         model_dir,
                                         qrnn, 
                                         emb_sz, 
                                         nh, nl,
                                         clas_drop_mult,
                                         fine_tune_lm,
                                         use_lm,cls_weights,
                                         deterministic,
                                         clas_lr_discount,
                                         lin_ftrs
                                        )

    ## Evaluation
    results = evaluation(y_true,y_pred)
    
    ## Log metrics
    for key, value in results.items():
        mlflow.log_metric(key, value)
        
    ## Save artifacts
    # pretrained lm
    mlflow.log_artifact(model_dir/f"{pretrained_fnames[0]}.pth")
    mlflow.log_artifact(model_dir/f"{pretrained_fnames[1]}.pkl")
    
    # fine tuned lm
    if fine_tune_lm:
         mlflow.log_artifact(model_dir / f'{lm_enc_finetuned}.pth')
    
    # classifier
    mlflow.log_artifact(model_dir / f'{model_name}_{name}.pth')
    
    # predicted text
    if p:
        mlflow.log_artifact('tmp/lm_predicted.txt')
        print('\n'+p)

F1 score
micro: 0.8941755537325677
macro: 0.8828724925971921

Precision score
micro: 0.8762057877813505
macro: 0.8639551825106716

Recall score
micro: 0.9128978224455612
macro: 0.9100219738655966

料金が高い 事 買 え る ▁ 。 ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ 朝 、 微 妙 だ エ チ ク ク ジ ト パ し 、 内部 の 年 、 特に 養成 行 ませ ならない 。 ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ ▁ 部 活 に


In [67]:
data_lm.train_ds.x[:5]

LMTextList (5 items)
[Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 朝 から 猛 烈 に 腹 が 痛 かった け ど 、 案 の 定 下 <unk> だった 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 軽 い 頭 痛 だ から 、 大 丈 夫 っ しょ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 下 <unk> って 、 英語 で なん て 言う ?, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ お 腹 が 痛 く て 起きた ら 、 下 <unk> になっていた ・ ・ ・ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ せ っ かく の スペイン な の に 風 邪 ひ いた ・ ・ ・ 。]...
Path: data/MedWeb

In [68]:
data_lm.train_ds.x.get(1)

Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 軽 い 頭 痛 だ から 、 大 丈 夫 っ しょ 。

In [69]:
train_df.head()

,0,1
0,Cold,風邪を引くと全身がだるくなる。
1,"Hayfever,Runnynose",花粉症の症状が出てきたのは久し振りだ。
2,"Hayfever,Fever,Runnynose",花粉症のせいでずっと微熱でぼーっとしてる。眠い。
3,Runnynose,薬飲んだけど鼻水おさまる気配なし
4,,ネパールに旅行に行った際に下痢になって大変だったよ。


In [72]:
t=sp['tokenizer']

In [73]:
tok=t.process_all(train_df[1])

In [74]:
tok

[['▁', '風', '邪', 'を', '引', 'く', 'と', '全', '身', 'が', 'だ', 'る', 'くなる', '。'],
 ['▁',
  '花',
  '粉',
  '症',
  'の',
  '症状',
  'が出',
  'てきた',
  'の',
  'は',
  '久',
  'し',
  '振り',
  'だ',
  '。'],
 ['▁',
  '花',
  '粉',
  '症',
  'の',
  'せい',
  'で',
  'ず',
  'っと',
  '微',
  '熱',
  'で',
  'ぼ',
  'ー',
  'っ',
  'として',
  'る',
  '。',
  '眠',
  'い',
  '。'],
 ['▁', '薬', '飲', 'ん', 'だけ', 'ど', '鼻', '水', 'お', 'さま', 'る', '気', '配', 'なし'],
 ['▁',
  'ネ',
  'パー',
  'ル',
  'に',
  '旅行',
  'に',
  '行った',
  '際に',
  '下',
  '痢',
  'になって',
  '大',
  '変',
  'だった',
  'よ',
  '。'],
 ['▁',
  '咳',
  'く',
  'らい',
  'で',
  '休',
  'む',
  'の',
  'は',
  'ゆ',
  'と',
  'り',
  'だけ',
  'だ',
  'ろ',
  '。',
  'ど',
  'ん',
  'な',
  'とき',
  'でも',
  '仕事',
  'に',
  'で',
  'る',
  'の',
  'は大',
  '事',
  'だ',
  '。'],
 ['▁', '鼻', 'づ', 'まり', 'で', '今日', 'は', '休', 'む', 'わ', 'ー'],
 ['▁', 'まさ', 'か', '花', '粉', '症', 'になると', 'は', '。'],
 ['▁',
  '熱',
  'は',
  '出て',
  'る',
  'け',
  'ど',
  'お',
  '腹',
  'に',
  '来',
  'る',
  '風',
  '邪',
  'じゃ',
  'な',
  'さ',
  'そう'

In [75]:
tok_v = t.process_all(valid_df[1])

In [87]:
data_lm = TextLMDataBunch.from_tokens(path=dataset_dir,trn_tok=tok,val_tok=tok_v,trn_lbls=[],val_lbls=[],vocab=sp['vocab'])

In [88]:
data_lm

TextLMDataBunch;
Train: LabelList
y: LMLabel (1920 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/MedWeb
x: LMTextList (1920 items)
[Text ▁ 風 邪 を 引 く と 全 身 が だ る くなる 。, Text ▁ 花 粉 症 の 症状 が出 てきた の は 久 し 振り だ 。, Text ▁ 花 粉 症 の せい で ず っと 微 熱 で ぼ ー っ として る 。 眠 い 。, Text ▁ 薬 飲 ん だけ ど 鼻 水 お さま る 気 配 なし, Text ▁ ネ パー ル に 旅行 に 行った 際に 下 <unk> になって 大 変 だった よ 。]...
Path: data/MedWeb;
Valid: LabelList
y: LMLabel (640 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/MedWeb
x: LMTextList (640 items)
[Text ▁ 旅行 に 行った ら 、 土 産 に イン フル も ら ってきた 。, Text ▁ き つ い 上 司 、 頭 痛 の 種, Text ▁ もう 誰 か 翻訳 し てくれ ない とき つ い 、 宇宙 の 言葉 です か って く らい 通じ なく て 頭 痛 してきた 。, Text ▁ イン フル 感染 の 危機 。, Text ▁ 鼻 づ まり が ひ ど い から ス ピー チ は 無 理 だ]...
Path: data/MedWeb;
Test: None

In [80]:
data_lm = TextLMDataBunch.from_df(path=dataset_dir,train_df=train_df.sample(frac=frac_ds,random_state=42),
                                      valid_df=valid_df,label_delim=',',**sp)

In [81]:
data_lm

TextLMDataBunch;
Train: LabelList
y: LMLabel (1920 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/MedWeb
x: LMTextList (1920 items)
[Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 朝 から 猛 烈 に 腹 が 痛 かった け ど 、 案 の 定 下 <unk> だった 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 軽 い 頭 痛 だ から 、 大 丈 夫 っ しょ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 下 <unk> って 、 英語 で なん て 言う ?, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ お 腹 が 痛 く て 起きた ら 、 下 <unk> になっていた ・ ・ ・ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ せ っ かく の スペイン な の に 風 邪 ひ いた ・ ・ ・ 。]...
Path: data/MedWeb;
Valid: LabelList
y: LMLabel (640 items)
[Category 0, Category 0, Category 0, Category 0, Category 0]...
Path: data/MedWeb
x: LMTextList (640 items)
[Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 旅行 に 行った ら 、 土 産 に イン フル も ら ってきた 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ き つ い 上 司 、 頭 痛 の 種, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ もう 誰 か 翻訳 し てくれ ない とき つ い 、 宇宙 の 言葉 です か って く らい 通じ なく て 頭 痛 してきた 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ イン フル 感染 の 危機 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 鼻 づ まり が ひ ど い から ス ピー チ は 無 理 だ]...
Path: data/MedWeb;
Tes

In [84]:
# load train, valid in df
train_df = pd.read_csv(dataset_dir/'train.csv',header=None)
valid_df = pd.read_csv(dataset_dir/'valid.csv',header=None)

In [85]:
train_df[0].apply(eval).head()

0                          [Cold]
1           [Hayfever, Runnynose]
2    [Hayfever, Fever, Runnynose]
3                     [Runnynose]
4                              []
Name: 0, dtype: object

In [86]:
train_df[0].apply(eval).values

array([list(['Cold']), list(['Hayfever', 'Runnynose']), list(['Hayfever', 'Fever', 'Runnynose']), list(['Runnynose']),
       ..., list(['Cold']), list(['Diarrhea']), list([]), list(['Cold'])], dtype=object)

In [89]:
data_clas_new = TextClasDataBunch.from_tokens(path=dataset_dir,trn_tok=tok,val_tok=tok_v,
                                              trn_lbls=train_df[0].apply(eval).values,
                                              val_lbls=valid_df[0].apply(eval).values,vocab=sp['vocab'])

In [90]:
data_clas_new

TextClasDataBunch;
Train: LabelList
y: MultiCategoryList (1920 items)
[MultiCategory Cold, MultiCategory Hayfever;Runnynose, MultiCategory Hayfever;Fever;Runnynose, MultiCategory Runnynose, MultiCategory ]...
Path: .
x: TextList (1920 items)
[Text ▁ 風 邪 を 引 く と 全 身 が だ る くなる 。, Text ▁ 花 粉 症 の 症状 が出 てきた の は 久 し 振り だ 。, Text ▁ 花 粉 症 の せい で ず っと 微 熱 で ぼ ー っ として る 。 眠 い 。, Text ▁ 薬 飲 ん だけ ど 鼻 水 お さま る 気 配 なし, Text ▁ ネ パー ル に 旅行 に 行った 際に 下 <unk> になって 大 変 だった よ 。]...
Path: data/MedWeb;
Valid: LabelList
y: MultiCategoryList (640 items)
[MultiCategory Influenza;Fever, MultiCategory , MultiCategory , MultiCategory Influenza;Fever, MultiCategory Runnynose]...
Path: .
x: TextList (640 items)
[Text ▁ 旅行 に 行った ら 、 土 産 に イン フル も ら ってきた 。, Text ▁ き つ い 上 司 、 頭 痛 の 種, Text ▁ もう 誰 か 翻訳 し てくれ ない とき つ い 、 宇宙 の 言葉 です か って く らい 通じ なく て 頭 痛 してきた 。, Text ▁ イン フル 感染 の 危機 。, Text ▁ 鼻 づ まり が ひ ど い から ス ピー チ は 無 理 だ]...
Path: data/MedWeb;
Test: None

In [92]:
data_clas_new.train_ds.y.c2i

{'Cold': 0,
 'Cough': 1,
 'Diarrhea': 2,
 'Fever': 3,
 'Hayfever': 4,
 'Headache': 5,
 'Influenza': 6,
 'Runnynose': 7}

In [97]:
data_clas_new.train_ds.x.vocab.itos

['<unk>',
 '<pad>',
 'xxbos',
 'xxfld',
 'xxunk',
 'xxpad',
 'の',
 '、',
 '。',
 '▁',
 'は',
 'が',
 'を',
 'に',
 '・',
 ')',
 '年',
 '(',
 'で',
 'と',
 '.',
 '」',
 'した',
 '「',
 '月',
 '▁"',
 'し',
 'から',
 '1',
 'である',
 '2',
 'する',
 'も',
 'や',
 '日',
 'ス',
 '3',
 'として',
 'た',
 'では',
 'る',
 '4',
 'された',
 '』',
 '大',
 'な',
 '『',
 '人',
 'ア',
 'い',
 'り',
 '5',
 'ク',
 'ト',
 'には',
 'している',
 'この',
 '7',
 'して',
 'という',
 '8',
 'リ',
 'ラ',
 'か',
 '6',
 '9',
 '中',
 '10',
 'その',
 '年に',
 'く',
 'また',
 'など',
 'て',
 '本',
 'ル',
 '市',
 '山',
 '-',
 '第',
 'され',
 'カ',
 'ド',
 '一',
 '上',
 '田',
 '国',
 'マ',
 '者',
 'ズ',
 '長',
 '地',
 'ている',
 'される',
 '下',
 'ら',
 'コ',
 '▁-',
 'タ',
 '川',
 'ナ',
 '日本',
 'されている',
 '12',
 '18',
 'フ',
 '家',
 '""',
 '名',
 'み',
 'ロ',
 'き',
 '線',
 '町',
 'エ',
 '戦',
 '水',
 '後',
 'となった',
 '回',
 '同',
 'によって',
 ',',
 'った',
 '法',
 '11',
 '部',
 '日に',
 '高',
 'ミ',
 'ム',
 '新',
 '子',
 'オ',
 'バ',
 '駅',
 'による',
 '前',
 'C',
 'ッ',
 ':',
 '生',
 '内',
 'ブ',
 '三',
 'レ',
 '化',
 'シ',
 'イ',
 'により',
 '数',
 'e',
 '小',
 '位',
 

In [94]:
data_clas

TextClasDataBunch;
Train: LabelList
y: MultiCategoryList (1920 items)
[MultiCategory Diarrhea, MultiCategory Headache, MultiCategory , MultiCategory Diarrhea, MultiCategory Cold]...
Path: data/MedWeb
x: TextList (1920 items)
[Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 朝 から 猛 烈 に 腹 が 痛 かった け ど 、 案 の 定 下 <unk> だった 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 軽 い 頭 痛 だ から 、 大 丈 夫 っ しょ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 下 <unk> って 、 英語 で なん て 言う ?, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ お 腹 が 痛 く て 起きた ら 、 下 <unk> になっていた ・ ・ ・ 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ せ っ かく の スペイン な の に 風 邪 ひ いた ・ ・ ・ 。]...
Path: data/MedWeb;
Valid: LabelList
y: MultiCategoryList (640 items)
[MultiCategory Influenza;Fever, MultiCategory , MultiCategory , MultiCategory Influenza;Fever, MultiCategory Runnynose]...
Path: data/MedWeb
x: TextList (640 items)
[Text ▁ xxbos ▁ xxfld ▁ 1 ▁ 旅行 に 行った ら 、 土 産 に イン フル も ら ってきた 。, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ き つ い 上 司 、 頭 痛 の 種, Text ▁ xxbos ▁ xxfld ▁ 1 ▁ もう 誰 か 翻訳 し てくれ ない とき つ い 、 宇宙 の 言葉 です か って く らい 通じ なく て 頭 痛 してきた 。, Text 

In [95]:
data_clas.train_ds.y.c2i

{'Cold': 0,
 'Cough': 1,
 'Diarrhea': 2,
 'Fever': 3,
 'Hayfever': 4,
 'Headache': 5,
 'Influenza': 6,
 'Runnynose': 7}